In [ ]:
#non riconosce la punteggiatura nella domanda, quindi da errore se lo scrivo nella risposta
import re
import ipywidgets as widgets
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class ManualLoader:
    def __init__(self, manual_paths):
        self.manuals = {}
        self.load_manuals(manual_paths)

    def load_manuals(self, paths):
        for name, path in paths.items():
            with open(path, 'r', encoding='utf-8') as file:
                content = file.read()
                # Dividi il contenuto in chunk usando '---' come delimitatore
                self.manuals[name] = self._split_into_chunks(content)

    def _split_into_chunks(self, text):
        # Splitting the text based on '---' delimiter
        return [chunk.strip() for chunk in text.split('---') if chunk.strip()]

class AssemblyAssistantBot:
    def __init__(self, manuals):
        self.manual_loader = ManualLoader(manuals)
        self.vectorizer = TfidfVectorizer()
        self._fit_vectorizer()

    def _fit_vectorizer(self):
        all_chunks = []
        for chunks in self.manual_loader.manuals.values():
            all_chunks.extend(chunks)
        self.vectorizer.fit(all_chunks)

    def handle_query(self, product, query):
        if product == "Giraffa":
            return self._handle_giraffa_query(query)
        else:
            return self._handle_other_products_query(product, query)

    def _handle_giraffa_query(self, query):
        chunks = self.manual_loader.manuals.get("Giraffa", [])
        if not chunks:
            return ["Manuale Giraffa non trovato."]
        
        query_vec = self.vectorizer.transform([query])
        results = []
        
        # Calcola la similarità e seleziona i chunk migliori
        chunk_vecs = self.vectorizer.transform(chunks)
        similarities = cosine_similarity(query_vec, chunk_vecs).flatten()
        
        for index, similarity in enumerate(similarities):
            if similarity > 0.25:  # Threshold per la similarità
                results.append(chunks[index])
        
        # Riduci la lunghezza delle risposte
        results = self._truncate_responses(results)
        
        return results if results else ["Nessuna informazione trovata per Giraffa."]

    def _handle_other_products_query(self, product, query):
        chunks = self.manual_loader.manuals.get(product, [])
        if not chunks:
            return ["Manuale non trovato."]
        
        query_vec = self.vectorizer.transform([query])
        results = []
        
        # Calcola la similarità e seleziona i chunk migliori
        chunk_vecs = self.vectorizer.transform(chunks)
        similarities = cosine_similarity(query_vec, chunk_vecs).flatten()
        
        for index, similarity in enumerate(similarities):
            if similarity > 0.25:  # Threshold per la similarità
                results.append(chunks[index])
        
        # Riduci la lunghezza delle risposte
        results = self._truncate_responses(results)
        
        return results if results else ["Nessuna informazione trovata."]

    def _truncate_responses(self, responses, max_length=500):
        # Truncare ogni risposta a una lunghezza massima
        truncated_responses = []
        for response in responses:
            if len(response) > max_length:
                truncated_responses.append(response[:max_length] + '...')
            else:
                truncated_responses.append(response)
        return truncated_responses

def on_submit(_):
    product = product_dropdown.value
    query = query_input.value
    if product and query:
        response = bot.handle_query(product, query)
        conversation_output.clear_output()
        with conversation_output:
            print(f"Tu: {query}")
            for info in response:
                print(f"Bot: {info}")
        query_input.value = ""
    else:
        with conversation_output:
            print("Bot: Seleziona un prodotto e inserisci una domanda.")

# Percorsi ai manuali
manuals = {
    "Treno Arancio": "C:\\Users\\marid\\OneDrive\\Desktop\\Progetto SF\\TrenoArancio.txt",
    "Treno Viola": "C:\\Users\\marid\\OneDrive\\Desktop\\Progetto SF\\TrenoViola.txt",
    "Giraffa": "C:\\Users\\marid\\OneDrive\\Desktop\\Progetto SF\\Giraffa.txt"
}

# Creazione del bot
bot = AssemblyAssistantBot(manuals)

# Menu a tendina per la selezione del prodotto (inizialmente popolato)
product_dropdown = widgets.Dropdown(
    options=[('Seleziona un prodotto', None)] + [(name, name) for name in manuals.keys()],
    value=None,
    description='Prodotto:',
)

# Campo di inserimento per la domanda
query_input = widgets.Text(
    value='',
    placeholder='Inserisci la tua domanda',
    description='Domanda:',
)

# Pulsante di invio
submit_button = widgets.Button(
    description='Invia',
    button_style='primary',
)
submit_button.on_click(on_submit)

# Finestra di conversazione
conversation_output = widgets.Output()

# Visualizza tutti i widget
display(product_dropdown, query_input, submit_button, conversation_output)


In [35]:
#Riconosce la punteggiatura nella domanda e non da errore
import re
import ipywidgets as widgets
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def preprocess_text(text):
    # Converti il testo in minuscolo, ma non rimuovere la punteggiatura
    return text.lower()

class ManualLoader:
    def __init__(self, manual_paths):
        self.manuals = {}
        self.load_manuals(manual_paths)

    def load_manuals(self, paths):
        for name, path in paths.items():
            with open(path, 'r', encoding='utf-8') as file:
                content = file.read()
                # Dividi il contenuto in chunk usando '---' come delimitatore
                self.manuals[name] = self._split_into_chunks(content)

    def _split_into_chunks(self, text):
        # Splitting the text based on '---' delimiter and preprocessing
        chunks = [preprocess_text(chunk.strip()) for chunk in text.split('---') if chunk.strip()]
        return chunks

class AssemblyAssistantBot:
    def __init__(self, manuals):
        self.manual_loader = ManualLoader(manuals)
        self.vectorizer = TfidfVectorizer()
        self._fit_vectorizer()

    def _fit_vectorizer(self):
        all_chunks = []
        for chunks in self.manual_loader.manuals.values():
            all_chunks.extend(chunks)
        self.vectorizer.fit(all_chunks)

    def handle_query(self, product, query):
        query = preprocess_text(query)
        if product == "Giraffa":
            return self._handle_giraffa_query(query)
        else:
            return self._handle_other_products_query(product, query)

    def _handle_giraffa_query(self, query):
        chunks = self.manual_loader.manuals.get("Giraffa", [])
        if not chunks:
            return ["Manuale Giraffa non trovato."]
        
        query_vec = self.vectorizer.transform([query])
        results = []
        
        # Calcola la similarità e seleziona i chunk migliori
        chunk_vecs = self.vectorizer.transform(chunks)
        similarities = cosine_similarity(query_vec, chunk_vecs).flatten()
        
        for index, similarity in enumerate(similarities):
            if similarity > 0.25:  # Threshold per la similarità
                results.append(chunks[index])
        
        # Riduci la lunghezza delle risposte
        results = self._truncate_responses(results)
        
        return results if results else ["Nessuna informazione trovata per Giraffa."]

    def _handle_other_products_query(self, product, query):
        chunks = self.manual_loader.manuals.get(product, [])
        if not chunks:
            return ["Manuale non trovato."]
        
        query_vec = self.vectorizer.transform([query])
        results = []
        
        # Calcola la similarità e seleziona i chunk migliori
        chunk_vecs = self.vectorizer.transform(chunks)
        similarities = cosine_similarity(query_vec, chunk_vecs).flatten()
        
        for index, similarity in enumerate(similarities):
            if similarity > 0.25:  # Threshold per la similarità
                results.append(chunks[index])
        
        # Riduci la lunghezza delle risposte
        results = self._truncate_responses(results)
        
        return results if results else ["Nessuna informazione trovata."]

    def _truncate_responses(self, responses, max_length=500):
        # Truncare ogni risposta a una lunghezza massima
        truncated_responses = []
        for response in responses:
            if len(response) > max_length:
                truncated_responses.append(response[:max_length] + '...')
            else:
                truncated_responses.append(response)
        return truncated_responses

def on_submit(_):
    product = product_dropdown.value
    query = query_input.value
    if product and query:
        response = bot.handle_query(product, query)
        conversation_output.clear_output()
        with conversation_output:
            print(f"Tu: {query}")
            for info in response:
                print(f"Bot: {info}")
        query_input.value = ""
    else:
        with conversation_output:
            print("Bot: Seleziona un prodotto e inserisci una domanda.")

# Percorsi ai manuali
manuals = {
    "Treno Arancio": "C:\\Users\\marid\\OneDrive\\Desktop\\Progetto SF\\TrenoArancio.txt",
    "Treno Viola": "C:\\Users\\marid\\OneDrive\\Desktop\\Progetto SF\\TrenoViola.txt",
    "Giraffa": "C:\\Users\\marid\\OneDrive\\Desktop\\Progetto SF\\Giraffa.txt"
}

# Creazione del bot
bot = AssemblyAssistantBot(manuals)

# Menu a tendina per la selezione del prodotto (inizialmente popolato)
product_dropdown = widgets.Dropdown(
    options=[('Seleziona un prodotto', None)] + [(name, name) for name in manuals.keys()],
    value=None,
    description='Prodotto:',
)

# Campo di inserimento per la domanda
query_input = widgets.Text(
    value='',
    placeholder='Inserisci la tua domanda',
    description='Domanda:',
)

# Pulsante di invio
submit_button = widgets.Button(
    description='Invia',
    button_style='primary',
)
submit_button.on_click(on_submit)

# Finestra di conversazione
conversation_output = widgets.Output()

# Visualizza tutti i widget
display(product_dropdown, query_input, submit_button, conversation_output)


Dropdown(description='Prodotto:', options=(('Seleziona un prodotto', None), ('Treno Arancio', 'Treno Arancio')…

Text(value='', description='Domanda:', placeholder='Inserisci la tua domanda')

Button(button_style='primary', description='Invia', style=ButtonStyle())

Output()